# Stable Diffusion Inference

## Setup

In [ ]:
!rm -r /content/sample_data

!pip install diffusers==0.11.1 transformers==4.25.1 scipy==1.9.3 ftfy==6.1.1
!pip install "ipywidgets>=7,<8"

# download validation set - designed for Google Colab
%cd /content/
for i in range(173):
  i = i + 1
  url = f"https://github.com/rmyj/txt2img/raw/main/assets/sd-v1-4.zip.{i:03}"
  !wget $url
# concatenate all part files to one zip
!cat /content/sd-v1-4.zip* > /content/sd-v1-4.zip
!rm /content/sd-v1-4.zip.*
# repair combination
!zip -FF /content/sd-v1-4.zip --out /content/sd-v1-4-full.zip
!rm /content/sd-v1-4.zip
# unzip combination
!unzip /content/sd-v1-4-full.zip
!rm /content/sd-v1-4-full.zip

from PIL import Image
from diffusers import StableDiffusionPipeline
import random
import uuid
import torch

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

# set variables for inference
model_id = "/content/sd-v1-4/"
pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe = pipe.to("cuda")

# to disable the safety checker
pipe.safety_checker = lambda images, clip_input: (images, False)

## Image Synthesis

In [ ]:
prompt = "an elephant in space, photorealistic, trending on artstation"

height = 512
width = 512

# can set seed here
seed = uuid.uuid4().int & (1<<64)-1
g_cuda = torch.Generator("cuda").manual_seed(seed)
        
print(f"{prompt}-{seed}")
image = pipe(prompt, height=height, width=width, num_inference_steps=74, guidance_scale=7.5, generator=g_cuda).images[0]

image